In [1]:
import os
import time
from PIL import Image
from PIL.ExifTags import TAGS
import glob

In [19]:
def get_exif_data(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    if exif_data:
        exif = {TAGS.get(tag, tag): value for tag, value in exif_data.items()}
        return exif
    else:
        return None

def get_photo_datetime(exif_data):
    # 'DateTimeOriginal' contains the date and time the photo was taken
    date_time_str = exif_data.get('DateTimeOriginal', None)
    if date_time_str:
        # Convert the string "YYYY:MM:DD HH:MM:SS" to "YYYY-MM-DD HH:MM:SS"
        date_time_str = date_time_str.replace(':', '-', 2)
        return date_time_str
    else:
        return None

def rename_to_epoch(image_path, processed_files, n):
    exif_data = get_exif_data(image_path)
    if not exif_data:
        print(f"GOPRO{n} - No EXIF metadata found for {image_path}")
        return

    date_time_str = get_photo_datetime(exif_data)
    if not date_time_str:
        print(f"GOPRO{n} - Date and time not found in metadata for {image_path}")
        return
    
    # Convert the date and time to a Unix timestamp (epoch time)
    epoch_time = int(time.mktime(time.strptime(date_time_str, "%Y-%m-%d %H:%M:%S")))
    
    # Get the directory and file extension
    directory, original_filename = os.path.split(image_path)
    file_extension = os.path.splitext(original_filename)[1]

    # Base filename using the epoch time
    new_filename = f"{epoch_time}{file_extension}"
    new_file_path = os.path.join(directory, new_filename)

    # Check if the file has already been processed (exists in processed files) or already exists
    if new_file_path in processed_files or os.path.exists(new_file_path):
        # print(f"Skipping '{image_path}' as it has already been renamed or a file with the name '{new_file_path}' exists.")
        return

    # Rename the file
    try:
        os.rename(image_path, new_file_path)
        # print(f"Renamed '{image_path}' to '{new_file_path}'")
        processed_files.add(new_file_path)  # Add to processed list
    except Exception as e:
        print(f"GOPRO{n} - Error renaming '{image_path}': {e}")

def missing_files(folder_path, n):
    # Get a list of all .JPG files in the folder
    files = [f for f in os.listdir(folder_path) if f.endswith('.JPG')]

    # Filter files to include only those with numeric filenames
    numeric_files = []
    for f in files:
        # Extract the filename without extension
        filename_without_ext = os.path.splitext(f)[0]
        
        # Check if the filename is entirely numeric
        if filename_without_ext.isdigit():
            numeric_files.append(f)
    
    # Ensure the list is not empty before proceeding
    if not numeric_files:
        print(f"GOPRO{n} - No numeric .JPG files found in the specified folder.")
        return None

    # Extract the numeric part of the filenames and sort them
    file_numbers = sorted([int(f.split('.')[0]) for f in numeric_files])

    # Initialize a list to store missing files
    missing_files = []

    # Check for missing numbers in the sequence
    for i in range(file_numbers[0], file_numbers[-1]):
        if i not in file_numbers:
            missing_files.append(i)

    # Output the result
    if missing_files:
        print(f"GOPRO{n} - Missing files: {', '.join([str(mf) + '.JPG' for mf in missing_files])}")
    else:
        print(f"GOPRO{n} - No missing files in the sequence.")
    return None


def process_folder(folder_path, n):
    # Search for all image files in the folder with common image extensions
    image_files = glob.glob(os.path.join(folder_path, "*.[jJpP][pPnN][gG]"))  # This includes .jpg, .png, .jpeg
    
    # Set to keep track of already processed files (renamed files)
    processed_files = set(glob.glob(os.path.join(folder_path, "*.[eE][pP][oO][cC][hH]*")))

    # Iterate over each image file and rename it based on EXIF metadata
    for image_path in image_files:
        rename_to_epoch(image_path, processed_files, n)

    missing_files(folder_path, n)


In [21]:
# Example usage
folder_path = r"C:\Users\maash\OneDrive - Delft University of Technology (1)\MDPCostaRica2024\Phase 2 - Execution\Results\GoPro\20241001"

for n in range(126, 139):
    new_folder_path = folder_path + '\\' + str(n) + 'GOPRO' 
    process_folder(new_folder_path, n)

GOPRO126 - No missing files in the sequence.
GOPRO127 - No missing files in the sequence.
GOPRO128 - No missing files in the sequence.
GOPRO129 - No missing files in the sequence.
